In [ ]:
# EDA Continued: Challenge Set 1 Review, Data Checking & Cleaning with Pandas and Viz Tools

Aka, learning how to always expect your data to have more problems.

**Learning Goals**:

1. Review core pandas methods and understand their application to challenge set 1
2. Understand basic methods for data quality checking and cleaning 
3. See examples of how to use visualization as an aid in exploring data quality 

In [ ]:
from __future__ import print_function, division

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import datetime

#### Exercise 1.1

- Open up a new IPython notebook
- Download a few MTA turnstile data files
- Read them into a pandas dataframe (pd.read_csv()), format the columns nicely, and display the first few rows.

In [ ]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [180505]
turnstiles_df = get_data(week_nums)

In [ ]:
turnstiles_df.head()

In [ ]:
# do the column names make any sense? sanity checking the column name
turnstiles_df.columns

In [ ]:
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [ ]:
turnstiles_df.columns

In [ ]:
# Three weeks of Data
#This is to make sure we have 3 weeks worth of data
turnstiles_df.DATE.value_counts().sort_index()

In [ ]:
from datetime import datetime as dt

In [ ]:
# Take the date and time fields into a single datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [ ]:
#need the below values to specify a specific turnstile. 

mask = ((turnstiles_df["C/A"] == "A002") & 
        (turnstiles_df["UNIT"] == "R051") & 
        (turnstiles_df["SCP"] == "02-00-00") & 
        (turnstiles_df["STATION"] == "59 ST"))
#mask, this returns true and false for all rows in the data frame. 
turnstiles_df[mask].head()

In [ ]:
mask = ((turnstiles_df["C/A"] == "R626") & 
(turnstiles_df["UNIT"] == "R062") & 
(turnstiles_df["SCP"] == "00-00-00") & 
(turnstiles_df["STATION"] == "CROWN HTS-UTICA"))
turnstiles_df[mask].head()

In [ ]:
turnstiles_df = turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])

In [ ]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

In [ ]:
# On 9/16, we seem to have two entries for same time.  Let's take a look
mask = ((turnstiles_df["C/A"] == "R504") & 
(turnstiles_df["UNIT"] == "R276") & 
(turnstiles_df["SCP"] == "00-00-01") & 
(turnstiles_df["STATION"] == "VERNON-JACKSON") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2016, 9, 16).date()))
turnstiles_df[mask].head()

In [ ]:
turnstiles_df.DESC.value_counts()

In [ ]:
# Get rid of the duplicate entry
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [ ]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
# check there are no other duplicates for all data sets 
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .EXITS.count()
 .reset_index()
 .sort_values("EXITS", ascending=False)).head(5)

In [ ]:
turnstiles_df[["PREV_DATE_TIME", "PREV_ENTRIES", "PREV_EXITS"]] = (turnstiles_df
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE_TIME", "ENTRIES", "EXITS"]
                                                       .transform(lambda grp: grp.shift(1)))

In [ ]:
# Drop the rows for the earliest date in the df
turnstiles_df.dropna(subset=["PREV_DATE_TIME"], axis=0, inplace=True)

In [ ]:
turnstiles_df[turnstiles_df["ENTRIES"] < turnstiles_df["PREV_ENTRIES"]].head()

In [ ]:
# Let's see how many stations have this problem

(turnstiles_df[turnstiles_df["ENTRIES"] < turnstiles_df["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size())

In [ ]:
def get_hourly_interval_counts(row, max_counter, column):
    counter = row[column] - row["PREV_"+column]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row[column], row["PREV_"+column])
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
_ = turnstiles_df.apply(get_hourly_interval_counts, axis=1, max_counter=1000000, column = "EXITS")

In [ ]:
def get_hourly_interval_counts(row, max_counter, column):
    counter = row[column] - row["PREV_"+column]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row[column], row["PREV_"+column])
        counter = min(row[column], row["PREV_"+column])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
turnstiles_df["INTERVAL_ENTRIES"] = turnstiles_df.apply(get_hourly_interval_counts, axis=1, max_counter=1000000, column="ENTRIES")
turnstiles_df["INTERVAL_EXITS"] = turnstiles_df.apply(get_hourly_interval_counts, axis=1, max_counter=1000000, column="EXITS")

In [ ]:
turnstiles_df["COMMUTE_INTERVAL"] = (turnstiles_df["DATE_TIME"]
                                                       .transform(lambda t: (t.hour >= 10 and
                                                                                 t.hour <= 14)+2*(t.hour >= 20)))

In [ ]:
turnstiles_df[turnstiles_df.COMMUTE_INTERVAL==1]["INTERVAL_ENTRIES"].max()

In [ ]:
turnstiles_df.loc[turnstiles_df.INTERVAL_ENTRIES==35112]

In [ ]:
turnstiles_df[turnstiles_df.DIVISION=="PTH"]["STATION"].unique()

In [ ]:
# turnstile_df = turnstiles_df.drop(turnstiles_df.STATION.isin[['NEWARK HW BMEBE', 'HARRISON', 'JOURNAL SQUARE', 'GROVE STREET',
#        'EXCHANGE PLACE', 'PAVONIA/NEWPORT', 'CITY / BUS',
#        'CHRISTOPHER ST', '9TH STREET', '14TH STREET', 'TWENTY THIRD ST',
#        'THIRTY ST', 'LACKAWANNA', 'THIRTY THIRD ST', 'NEWARK BM BW',
#        'NEWARK C', 'NEWARK HM HE', 'PATH WTC 2', 'PATH NEW WTC']])

newList = ['NEWARK HW BMEBE', 'HARRISON', 'JOURNAL SQUARE', 'GROVE STREET', 'EXCHANGE PLACE', 'PAVONIA/NEWPORT', 'CITY / BUS', 'LACKAWANNA', 'NEWARK BM BW',
       'NEWARK C', 'NEWARK HM HE']
new_turnstile_df = turnstiles_df[~turnstiles_df['STATION'].isin(newList)]

In [ ]:
new_turnstile_df[new_turnstile_df.DIVISION=="PTH"]["STATION"].unique()

In [ ]:
new_turnstile_df[new_turnstile_df.COMMUTE_INTERVAL==1]["INTERVAL_ENTRIES"].max()

In [ ]:
new_turnstile_df.loc[new_turnstile_df.INTERVAL_ENTRIES==3006]

**VISUALIZATIONS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

In [ ]:
# first get daily entries by station
stations_Interval_Entries = \
(new_turnstile_df[new_turnstile_df["COMMUTE_INTERVAL"] == 1].groupby(['STATION'])['INTERVAL_ENTRIES'].mean()
                 .reset_index()).sort_values(by="INTERVAL_ENTRIES", ascending=False)  

stations_Interval_Exits = \
(new_turnstile_df[new_turnstile_df["COMMUTE_INTERVAL"] == 2].groupby(['STATION'])['INTERVAL_EXITS'].mean()
                 .reset_index()).sort_values(by="INTERVAL_EXITS", ascending=False) 


stations_Interval_Entries.describe()
#stations_Interval_Exits.head()



In [ ]:
stations_Interval_Entries = \
(new_turnstile_df.groupby(['STATION'])['INTERVAL_ENTRIES'].mean()
                 .reset_index()).sort_values(by="INTERVAL_ENTRIES", ascending=False)  

stations_Interval_Exits = \
(new_turnstile_df.groupby(['STATION'])['INTERVAL_EXITS'].mean()
                 .reset_index()).sort_values(by="INTERVAL_EXITS", ascending=False) 

In [ ]:
sns.distplot(new_turnstile_df['INTERVAL_ENTRIES']
             [new_turnstile_df['INTERVAL_ENTRIES'] < 10000])



In [ ]:
sns.distplot(new_turnstile_df['INTERVAL_EXITS']
             [new_turnstile_df['INTERVAL_EXITS'] < 10000])

In [ ]:
# then get top 10 stations by daily volume 
# (sum across all days is a reasonable way to define this)
top10_stations = \
    (stations_daily.groupby(['STATION'])['DAILY_ENTRIES'].sum()
                   .reset_index()
                   .sort_values(by='DAILY_ENTRIES',ascending=False) 
                   .STATION.head(10))

top10_stations

In [ ]:
# next create a new df that filters the stations daily data down
# to the top 10 stations
stations_daily_top10 = \
    stations_daily[stations_daily['STATION'].isin(top10_stations)]

In [ ]:
# use seaborn to create a boxplot by station
sns.boxplot('DAILY_ENTRIES', 'STATION', data=stations_daily_top10)